In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [2]:
'''
Item-Item Collaborative Filtering (|N|=2)
|N| = 2 represents the size of the set N.
'''

# The set N refers to the neighborhood or set of nearest
# neighbors of an item in an item-item collaborative filtering
# approach. In other words, for a particular item, the collaborative
# filtering  algorithm will select the two most similar items to
# form the neighborhood. These two items will then be used to
# make recommendations or provide insights.

N = 2

In [3]:
# Initialize toy matrix with ratings for testing purposes
A = np.array(
     [[ 1,  0,  3,  0,  0,  5,  0,  0,  5,  0,  4,  0],
      [ 0,  0,  5,  4,  0,  0,  4,  0,  0,  2,  1,  3],
      [ 2,  4,  0,  1,  2,  0,  3,  0,  4,  3,  5,  0],
      [ 0,  2,  4,  0,  5,  0,  0,  4,  0,  0,  2,  0],
      [ 0,  0,  4,  3,  4,  2,  0,  0,  0,  0,  2,  5],
      [ 1,  0,  3,  0,  3,  0,  0,  2,  0,  0,  4,  0]])

rows = len(A)
cols = len(A[1])

In [4]:
# Initilize a list for as many items we've got in
# order to calculate the average ratings for them
item_Avg = [0] * A.shape[0]

# Iterate through the ratings matrix and calculate
# the average rating for each item available
for i in range(rows):
  item_Avg[i] = 0
  positive_Counter = 0
  for j in range(cols):
    if A[i][j] != 0:
      item_Avg[i] += A[i][j]
      positive_Counter += 1
  item_Avg[i] = round(item_Avg[i] / positive_Counter, 3)
# Print each items average
  print(item_Avg[i])

3.6
3.167
3.0
3.4
3.333
2.6


In [5]:
# Initialize an empty two-dimensional array that will
# contain the substractions of users ratings and items averages
minus_Array = [[0 for j in range(cols)] for i in range(rows)]

# Subtract the items average from each users rating
for i in range(len(minus_Array)):
  for j in range(len(minus_Array[0])):
    if A[i][j] != 0:
      minus_Array[i][j] = round(A[i][j] - item_Avg[i], 3)

# Print the updated array that's ready for cosine similarity
for row in minus_Array:
    print(row)

[-2.6, 0, -0.6, 0, 0, 1.4, 0, 0, 1.4, 0, 0.4, 0]
[0, 0, 1.833, 0.833, 0, 0, 0.833, 0, 0, -1.167, -2.167, -0.167]
[-1.0, 1.0, 0, -2.0, -1.0, 0, 0.0, 0, 1.0, 0.0, 2.0, 0]
[0, -1.4, 0.6, 0, 1.6, 0, 0, 0.6, 0, 0, -1.4, 0]
[0, 0, 0.667, -0.333, 0.667, -1.333, 0, 0, 0, 0, -1.333, 1.667]
[-1.6, 0, 0.4, 0, 0.4, 0, 0, -0.6, 0, 0, 1.4, 0]


In [ ]:
# Convert the 2D matrix filled with many zeros into
# a sparse matrix representation in Compressed Sparse
# Row (CSR) format for storage efficiency storage and
# easier computation on sparse matrices
A_sparse = sparse.csr_matrix(minus_Array)

# Calculate all the pairwise cosine similarities
# between each row of the 2D matrix
similarities_sparse = round(cosine_similarity(A_sparse, dense_output = False), 4)

# Prints For Testing Purposes

# # Cosine Similarities Output
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

# # Extract the similarity value from similarities_sparse
# similarity_ij = similarities_sparse[0, 1]

# # Print the similarity value
# print(f"The similarity between row {i} and row {j} is: {similarity_ij}")

In [ ]:
# max_similarities = {}

# # Iterate through the cosine similarities
# for i in range(similarities_sparse.shape[0]):
#     for j in range(i+1, similarities_sparse.shape[1]):
#         similarity_value = similarities_sparse[i, j]

#         if similarity_value < 0.99:
#             pair = (i, j)
#             if pair not in max_similarities or similarity_value > max_similarities[pair]:
#                 max_similarities[pair] = similarity_value

In [13]:
max_similarities = {}

# Iterate through the cosine similarities
for i in range(similarities_sparse.shape[0]):
    row_similarities = []
    for j in range(similarities_sparse.shape[1]):
        if i != j:
            similarity_value = similarities_sparse[i, j]
            row_similarities.append((j, similarity_value))

    # Sort the row similarities by similarity value in descending order
    row_similarities.sort(key=lambda x: x[1], reverse=True)

    # Keep only the top two maximum similarities for the row
    top_2_similarities = row_similarities[:2]

    # Extract the row indices (j) and similarity values for the row
    top_2_indices = [pair[0] for pair in top_2_similarities]
    top_2_values = [pair[1] for pair in top_2_similarities]

    # Store the top two maximum similarities (with row indices) in a list for the current row
    max_similarities[i] = list(zip(top_2_indices, top_2_values))

    # # Print the top two maximum similarities for the row
    # print(f"Top two maximum similarities for row {i}:")
    # for j, similarity in zip(top_2_indices, top_2_values):
    #     print(f"Row {j}: {similarity}")
    # print()

# Print the dictionary of top two maximum similarities for each row
print("Dictionary of top two maximum similarities for each row:")
for row, similarities in max_similarities.items():
    print(f"Row {row}: {similarities}")

Dictionary of top two maximum similarities for each row:
Row 0: [(5, 0.587), (2, 0.414)]
Row 1: [(3, 0.468), (4, 0.3989)]
Row 2: [(5, 0.5064), (0, 0.414)]
Row 3: [(1, 0.468), (4, 0.4588)]
Row 4: [(3, 0.4588), (1, 0.3989)]
Row 5: [(0, 0.587), (2, 0.5064)]


## Testing

In [14]:
# Lets say we want to calculate the rating of movie 1 for user 5

user = 4
movie = 0

# Get the list of max similarities for row 0
movie_0_similarities = max_similarities[movie]

# Unpack the first element (column index, similarity value)
column_index_1, similarity_value_1 = movie_0_similarities[0]

# Unpack the second element (column index, similarity value)
column_index_2, similarity_value_2 = movie_0_similarities[1]

print(column_index_1)  # Output: 5
print(similarity_value_1)  # Output: 0.587
print(column_index_2)  # Output: 2
print(similarity_value_2)  # Output: 0.414

5
0.587
2
0.414


In [17]:
# Prediction
prediction = round(((similarity_value_1 * A[column_index_1][user]) + (similarity_value_2 * A[column_index_2][user])) / (similarity_value_1 + similarity_value_2), 3)

In [18]:
prediction

2.586